<a href="https://colab.research.google.com/github/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_seguradora/analise_exploratoria_seguro_saude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Exploratória de Dados - Seguro Saúde


# Tópicos

1. Introdução e Preparação Inicial dos Dados
2. Limpeza e Pré-processamento de Dados
3. Análises
4. Insights e Conclusões


## 1. Introdução

- Objetivo dessa análise é identificar padrões de tendências entre os segurados e sua relação com as depesas.

- Os dados são de um dataset retirado do Kaggle, com o nome originalmente "[US Health Insurance Dataset](https://www.kaggle.com/datasets/teertha/ushealthinsurancedataset/data)" ele contém dados de segurados em um plano de saúde e algumas de suas características.

## 2. Carregamento e Inspeção Inicial

In [2]:
# Importando bibliotecas necessárias
import pandas as pd
import kagglehub

In [3]:
# Download versão mais atual
path = kagglehub.dataset_download("teertha/ushealthinsurancedataset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ushealthinsurancedataset


In [4]:
# Carrega arquivo baixado
file_path = '/kaggle/input/ushealthinsurancedataset/insurance.csv'
df = pd.read_csv(file_path)

In [5]:
#Verificando as primeiras linhas
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [6]:
#Verificando as últimas linhas
df.tail()

,age,sex,bmi,children,smoker,region,charges
1333,50,male,30.97,3,no,northwest,10600.5483
1334,18,female,31.92,0,no,northeast,2205.9808
1335,18,female,36.85,0,no,southeast,1629.8335
1336,21,female,25.80,0,no,southwest,2007.9450
1337,61,female,29.07,0,yes,northwest,29141.3603


In [7]:
#Verificando informações do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [10]:
# Estastisticas categoricas
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [9]:
# Estatisticas não categoricas
df.describe(include='object')

,sex,smoker,region
count,1338,1338,1338
unique,2,2,4
top,male,no,southeast
freq,676,1064,364
